In [19]:
import mysql.connector
import requests
import urllib
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

In [20]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password=os.getenv("SQL_PASSWORD"),
  database="nfl"
)
cursor = mydb.cursor()

In [21]:
cursor.execute('SELECT * FROM stadiums')


In [22]:
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[i[0] for i in cursor.description])

In [ ]:


# USGS Elevation Point Query Service
#url = r'https://nationalmap.gov/epqs/pqs.php?'
#new 2023:
url = r'https://epqs.nationalmap.gov/v1/json?'

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
                
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Feet'
        }
        
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        print(result.text)  # print the response text
        try:
            elevations.append(result.json()['value'])
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for {lat}, {lon}")
            elevations.append(None)

    df['altitude'] = elevations

elevation_function(df, 'latitude', 'longitude')
df.head()

In [27]:
for i,row in df.iterrows():
    altitude = row['altitude'] if pd.notna(row['altitude']) else None
    name = row['name'] if pd.notna(row['name']) else None
    sql = "UPDATE stadiums SET altitude = %s WHERE name = %s"
    data = (altitude, name)
    cursor.execute(sql,data)

mydb.commit()

if (mydb.is_connected()):
    cursor.close()
    mydb.close()
    print("MySQL connection is closed")

MySQL connection is closed
